In [1]:
import pandas as pd 
import json 
import shapefile
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import os 
import json
import numpy as np

def isNested(entry): 
    try: 
        sum(entry)
        return 0 
    except: 
        return 1

def readInJson(fileName):
    try: 
        df = pd.read_json(fileName)
    except: 
        handle = json.load(open(fileName))
        df = pd.DataFrame(handle['features'])
        
    return df

In [2]:
lonlatSuburb = pd.read_csv('lonlatSuburb.csv')
vicPostcodes = pd.read_csv('vicPostcodes.csv')

lonlatSuburb['lat'] = lonlatSuburb.lat.apply(lambda x: round(x, 4))
lonlatSuburb['long'] = lonlatSuburb.long.apply(lambda x: round(x, 4))

vicPostcodes['lat'] = vicPostcodes.lat.apply(lambda x: round(x, 6))
vicPostcodes['long'] = vicPostcodes.long.apply(lambda x: round(x, 6))

In [3]:
garbageCollectionDate = sorted([i for i in os.listdir() if i.endswith('json')])

garbageCollectionDate[15]

'WestWimmeraShireCouncil-garbage-collection-zones.json'

In [4]:
readInJson(garbageCollectionDate[15]).head(2)

,type,id,geometry,geometry_name,properties
0,Feature,ckan_e77b6c07_39f2_454d_ae72_4976cab1dfb3.1,"{'type': 'MultiPolygon', 'coordinates': [[[[14...",geom,"{'id': 0, 'route': 'Apsley', 'rub_day': 'Wedne..."
1,Feature,ckan_e77b6c07_39f2_454d_ae72_4976cab1dfb3.2,"{'type': 'MultiPolygon', 'coordinates': [[[[14...",geom,"{'id': 0, 'route': 'Kaniva', 'rub_day': 'Wedne..."


In [21]:
df = readInJson(garbageCollectionDate[15])

df['geo'] = df.geometry.apply(lambda x: x['coordinates'][0][0])

df = df.explode('geo')

df['geo_nested'] = df.geo.apply(lambda x: isNested(x))
df['lat'] = df.geo.apply(lambda x: round(x[0], 4))
df['long'] = df.geo.apply(lambda x: round(x[1], 4))

df.drop(['geo','id', 'geometry', 'geometry_name',
         'geo_nested', 'type'], axis = 1, inplace = True)

In [16]:
df.iloc[0,0]

{'id': 0,
 'route': 'Apsley',
 'rub_day': 'Wednesday',
 'rub_start': '2016-03-02',
 'rec_day': 'Wednesday',
 'rec_start': '2016-03-02',
 'info_url': 'http://www.westwimmera.vic.gov.au/waste-management',
 'rub_weeks': 1,
 'rec_weeks': 2}

In [22]:
def collectionFrequency(row, wasteType): 
    
    rub_colDay = row[f'{wasteType}_day']
    rub_colFreq = row[f'{wasteType}_weeks']
    
    if rub_colFreq == '2' or rub_colFreq == 2: 
        rub_colFreq = 'Every Second '
    else: 
        rub_colFreq = 'Every '
        
    if rub_colDay in [None, 'None']:
        rub_colDay = ''
    elif rub_colFreq == [None, 'None']: 
        rub_colFreq = ''
        
    returningStr = rub_colFreq + rub_colDay
    
    if len(returningStr.split())>1:
         return returningStr
    else: 
        return 'Unavailable'

df['landfill_colDay'] = df.properties.apply(lambda x: collectionFrequency(x, 'rub'))
df['recyclable_colDay'] = df.properties.apply(lambda x: collectionFrequency(x, 'rec'))

df.drop('properties', axis = 1, inplace = True)

In [23]:
df

,lat,long,landfill_colDay,recyclable_colDay
0,141.0638,-36.9568,Every Wednesday,Every Second Wednesday
0,141.0651,-36.9569,Every Wednesday,Every Second Wednesday
0,141.0658,-36.9569,Every Wednesday,Every Second Wednesday
0,141.0674,-36.9570,Every Wednesday,Every Second Wednesday
0,141.0815,-36.9583,Every Wednesday,Every Second Wednesday
...,...,...,...,...
10,141.0838,-36.9576,Every Wednesday,Unavailable
10,141.0838,-36.9576,Every Wednesday,Unavailable
10,141.0837,-36.9584,Every Wednesday,Unavailable
10,141.0837,-36.9584,Every Wednesday,Unavailable


In [24]:
lonlatSuburb.merge(df)[['locality', 'long', 'lat', 'landfill_colDay', 'recyclable_colDay']]

,locality,long,lat,landfill_colDay,recyclable_colDay


In [25]:
ww = lonlatSuburb.merge(df)[['locality', 'long', 'lat', 'landfill_colDay', 'recyclable_colDay']]\
.sort_values(['locality', 'landfill_colDay', 'recyclable_colDay'])\
        .drop_duplicates(['locality','recyclable_colDay', 'landfill_colDay', 'recyclable_colDay'])\
        .sort_values(['locality'])\
.merge(vicPostcodes[['locality', 'postcode']])\
.sort_values(['postcode', 'landfill_colDay', 'recyclable_colDay'])\
.drop_duplicates(subset = ['postcode', 'landfill_colDay', 'recyclable_colDay'])\
.rename(columns = dict(zip(['landfill_colDay', 'landfill_colDay', 'recyclable_colDay'], 
                           ['landfill', 'landfill', 'recyclable'])))\
.drop(['long', 'lat', 'locality'], axis = 1)\
.melt(id_vars = 'postcode')\
.rename(columns = {'variable':'binInfo_bin_type', 'value':'collection_time'})\
.drop_duplicates(['postcode', 'binInfo_bin_type'])

ww.to_csv('../GBD_wwa.csv', index = False)